In [2]:
from enums import CollectionNames, EmbeddingModels
from qdrant_client.models import Filter, FieldCondition, MatchValue

from retrieval_util import (
    get_qdrant_client,
    get_embeddings_function,
    retrieve_run_dict_with_documents_with_scores,
    create_llm_for_self_query_retriever
)

import os
import json
import logging
from ranx import Qrels, Run, evaluate, compare

os.environ[
    "TOKENIZERS_PARALLELISM"] = "false"  # Verhindert Fehler beim Erzeugen von Embeddings (Tritt sporadisch auf, konnte ich nicht gezielt reproduzieren)
logging.basicConfig(level=logging.INFO)

In [3]:
# Konstanten
MAX_DOCUMENTS = 10
BASE_PATH = "./mixedbread_retrieval_results_AT_10"
QDRANT_URL = "http://localhost:6333"
LLM = create_llm_for_self_query_retriever()
FRAGEN = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}
METRICS = ["ndcg@10", "mrr@10", "map@10", "precision@10", "recall@10", "hits@10", "hit_rate@10"]


# Alle relevanten Filter für die Fragen
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )


FILTERS = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}

In [4]:
token_based_qrels_dict = {
    "q_1": {
        "d_1021": 10,
        "d_1022": 1
    },
    "q_2": {
        "d_1301": 10
    },
    "q_3": {
        "d_1476": 10,
        "d_1480": 10,
        "d_1481": 2
    },
    "q_4": {
        "d_1498": 10,
        "d_1499": 10,
        "d_1500": 2
    },
    "q_5": {
        "d_1482": 10,
        "d_1483": 10,
        "d_1484": 10,
        "d_1485": 10
    },
    "q_6": {
        "d_18": 10
    },
    "q_7": {
        "d_1192": 10,
        "d_1193": 8
    },
    "q_8": {
        "d_1530": 10,
        "d_1553": 10,
        "d_1554": 10,
        "d_1555": 10
    },
    "q_9": {
        "d_704": 10
    },
    "q_10": {
        "d_14": 10,
        "d_15": 10
    }
}

TOKEN_BASED_QRELS = Qrels(token_based_qrels_dict)

In [5]:
recursive_qrels_dict = {
    "q_1": {
        "d_1371": 10,
        "d_1370": 1
    },
    "q_2": {
        "d_1866": 10
    },
    "q_3": {
        "d_2292": 10,
        "d_2293": 2,
        "d_2286": 10
    },
    "q_4": {
        "d_2313": 10,
        "d_2314": 10,
        "d_2315": 2
    },
    "q_5": {
        "d_2290": 4,
        "d_2295": 10,
        "d_2296": 10,
        "d_2297": 10,
        "d_2298": 10
    },
    "q_6": {
        "d_25": 10
    },
    "q_7": {
        "d_1586": 10
    },
    "q_8": {
        "d_2388": 10,
        "d_2389": 10,
        "d_2390": 10,
        "d_2356": 8,
        "d_2357": 8
    },
    "q_9": {
        "d_946": 10
    },
    "q_10": {
        "d_19": 10
    }
}

RECURSIVE_QRELS = Qrels(recursive_qrels_dict)

In [6]:
def save_results_to_file(file_path, results_dict):
    try:
        with open(file_path, "w") as file:
            json.dump(results_dict, file, indent=4)
        print(f"Ergebnisse wurden erfolgreich in {file_path} gespeichert.")
    except Exception as e:
        print(f"Fehler beim Speichern der Datei: {e}")

In [7]:
embeddings = get_embeddings_function(model_name=EmbeddingModels.MIXEDBREAD.value)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-16 13:15:15,009 [INFO] PyTorch version 2.6.0 available.
2025-03-16 13:15:15,248 [INFO] Use pytorch device_name: mps
2025-03-16 13:15:15,249 [INFO] Load pretrained SentenceTransformer: mixedbread-ai/deepset-mxbai-embed-de-large-v1


In [8]:
token_based_client = get_qdrant_client(embeddings=embeddings,
                                       collection_name=CollectionNames.MIXEDBREAD_TOKEN_BASED.value,
                                       qdrant_url=QDRANT_URL)

recursive_client = get_qdrant_client(embeddings=embeddings, collection_name=CollectionNames.MIXEDBREAD_RECURSIVE.value,
                                     qdrant_url=QDRANT_URL)

2025-03-16 13:15:19,778 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-16 13:15:19,785 [INFO] HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks "HTTP/1.1 200 OK"
2025-03-16 13:15:20,082 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-16 13:15:20,086 [INFO] HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks "HTTP/1.1 200 OK"


# Token-Basierte Collections Retrieval Evaluation

In [9]:
token_based_result_no_filters = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                             fragen_dict=FRAGEN, mode="no_filters",
                                                                             max_documents=MAX_DOCUMENTS)

token_based_result_optimal_filters = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                                  fragen_dict=FRAGEN,
                                                                                  mode="optimal_filters",
                                                                                  filter_dict=FILTERS,
                                                                                  max_documents=MAX_DOCUMENTS)

token_based_result_self_query_retriever = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                                       fragen_dict=FRAGEN,
                                                                                       mode="self_query_retriever",
                                                                                       llm=LLM,
                                                                                       search_kwargs={
                                                                                           "k": MAX_DOCUMENTS})

2025-03-16 13:15:20,253 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:15:20,355 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:15:20,439 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:15:20,489 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:15:20,598 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:15:20,677 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_de

In [10]:
save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_no_filters.json",
                     results_dict=token_based_result_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_optimal_filters.json",
                     results_dict=token_based_result_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_self_query_retriever.json",
                     results_dict=token_based_result_self_query_retriever)

Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/token_based_result_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/token_based_result_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/token_based_result_self_query_retriever.json gespeichert.


In [11]:
run_token_based_result_no_filters = Run(token_based_result_no_filters, name="token_based_result_no_filters")
run_token_based_result_optimal_filters = Run(token_based_result_optimal_filters,
                                             name="token_based_result_optimal_filters")
run_token_based_result_self_query_retriever = Run(token_based_result_self_query_retriever,
                                                  name="token_based_result_self_query_retriever")

In [12]:
token_based_scores_no_filters = evaluate(qrels=TOKEN_BASED_QRELS, run=run_token_based_result_no_filters,
                                         metrics=METRICS)
token_based_scores_optimal_filters = evaluate(qrels=TOKEN_BASED_QRELS, run=run_token_based_result_optimal_filters,
                                              metrics=METRICS)
token_based_scores_self_query_retriever = evaluate(qrels=TOKEN_BASED_QRELS,
                                                   run=run_token_based_result_self_query_retriever, metrics=METRICS)

In [13]:
save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_no_filters.json",
                     results_dict=token_based_scores_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_optimal_filters.json",
                     results_dict=token_based_scores_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_self_query_retriever.json",
                     results_dict=token_based_scores_self_query_retriever)

Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/token_based_scores_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/token_based_scores_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/token_based_scores_self_query_retriever.json gespeichert.


In [14]:
report = compare(
    qrels=TOKEN_BASED_QRELS,
    runs=[run_token_based_result_no_filters, run_token_based_result_optimal_filters,
          run_token_based_result_self_query_retriever],
    metrics=METRICS
)

In [15]:
print(report)

#    Model                                      NDCG@10    MRR@10    MAP@10    P@10    Recall@10    Hits@10    Hit Rate@10
---  ---------------------------------------  ---------  --------  --------  ------  -----------  ---------  -------------
a    token_based_result_no_filters                0.359     0.433     0.308    0.13        0.408        1.3            0.5
b    token_based_result_optimal_filters           0.48      0.52      0.403    0.18        0.7          1.8            0.7
c    token_based_result_self_query_retriever      0.417     0.453     0.336    0.15        0.608        1.5            0.7


In [16]:
with open(f"{BASE_PATH}/token_report@{MAX_DOCUMENTS}.txt", "w") as file:
    file.write(str(report))

# Recursive-Collections Retrieval Evaluation

In [17]:
recursive_result_no_filters = retrieve_run_dict_with_documents_with_scores(client=recursive_client, fragen_dict=FRAGEN, mode="no_filters", max_documents=MAX_DOCUMENTS)
recursive_result_optimal_filters = retrieve_run_dict_with_documents_with_scores(client=recursive_client, fragen_dict=FRAGEN, mode="optimal_filters", filter_dict=FILTERS, max_documents=MAX_DOCUMENTS)
recursive_result_self_query_retriever = retrieve_run_dict_with_documents_with_scores(client=recursive_client, fragen_dict=FRAGEN, mode="self_query_retriever", llm=LLM, search_kwargs={"k": MAX_DOCUMENTS})

2025-03-16 13:16:45,238 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:16:45,290 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:16:45,334 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:16:45,378 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:16:45,439 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:16:45,479 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxba

In [18]:
save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_no_filters.json", results_dict=recursive_result_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_optimal_filters.json", results_dict=recursive_result_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_self_query_retriever.json", results_dict=recursive_result_self_query_retriever)

Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/recursive_result_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/recursive_result_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/recursive_result_self_query_retriever.json gespeichert.


In [19]:
run_recursive_result_no_filters = Run(recursive_result_no_filters, name="recursive_result_no_filters")
run_recursive_result_optimal_filters = Run(recursive_result_optimal_filters, name="recursive_result_optimal_filters")
run_recursive_result_self_query_retriever = Run(recursive_result_self_query_retriever, name="recursive_result_self_query_retriever")

In [20]:
recursive_scores_no_filters = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_no_filters, metrics=METRICS)
recursive_scores_optimal_filters = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_optimal_filters, metrics=METRICS)
recursive_scores_self_query_retriever = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_self_query_retriever, metrics=METRICS)

In [21]:
save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_no_filters.json", results_dict=recursive_scores_no_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_optimal_filters.json", results_dict=recursive_scores_optimal_filters)
save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_self_query_retriever.json", results_dict=recursive_scores_self_query_retriever)

Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/recursive_scores_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/recursive_scores_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./mixedbread_retrieval_results_AT_10/recursive_scores_self_query_retriever.json gespeichert.


In [22]:
report = compare(
    qrels=RECURSIVE_QRELS,
    runs=[run_recursive_result_no_filters, run_recursive_result_optimal_filters, run_recursive_result_self_query_retriever],
    metrics=METRICS
)

In [23]:
print(report)

#    Model                                    NDCG@10    MRR@10    MAP@10    P@10    Recall@10    Hits@10    Hit Rate@10
---  -------------------------------------  ---------  --------  --------  ------  -----------  ---------  -------------
a    recursive_result_no_filters                0.439     0.483     0.403    0.17        0.547        1.7            0.6
b    recursive_result_optimal_filters           0.534     0.544     0.506    0.2         0.7          2              0.7
c    recursive_result_self_query_retriever      0.469     0.494     0.414    0.18        0.647        1.8            0.7


In [24]:
with open(f"{BASE_PATH}/recursive_report@{MAX_DOCUMENTS}.txt", "w") as file:
    file.write(str(report))

Die Ergebnisse des mixedbread-Modells sind sowohl bei den Token-Basierten Chunks als auch bei den Rekursiven Chunks besser.

Hier zeigt sich also, dass das auf die deutsche Sprache spezialisierte Modell besser mit der Datenbasis performt.<br/>

Ein reiner Vergleich auf basis der Metriken zeigt nicht, wie diese Unterschiede zwischen den Modellen zustande kommen.<br/>
Dafür müssen auch die Ergebnisse der Abrufe betrachtet werden. Dort sind teilweise andere Dokumente gefunden worden und die Scores unterscheiden sich.<br/>
Aufgrund der unterschiedlichen Scores entstehen unterschiedliche Rankings, die sich auf die rang-basierte Metriken auswirken.<br/>

Bei beiden Modellen sind die Metriken besser mit den rekursiven Chunks als mit den Token-Basierten Chunks.<br/>

In allen Fällen schneidet der Ansatz mit den manuell und optimal gesetzten Filtern am besten ab.<br/>
Der SelfQueryRetriever schneidet minimal besser als der Ansatz ohne Filter ab.<br/>
Der Ansatz ohne Filter liefert die schlechtesten Ergebnisse.<br/>

<br/>
Aufgrund der mangelnden Vergleichbarkeit werden die Ergebnisse des SelfQueryRetrievers nicht weiter betrachtet.<br/>
Grund sind die fehlenden Filter für Fragen zu Chunks, die in den Metadaten für die relevanten Felder den Wert "Alle" haben.<br/>
Zudem sind die umformulierten Fragen nicht identisch mit den ursprünglichen Fragen (Hier als vereinfachte Fragen bezeichnet und in dem Orner 6-Evaluation-Retrieval-Vereinfachte-Fragen evaluiert).***<br/>

In [25]:
frage_6_vereinfacht = "Wie viele Seiten muss ich in meiner Bachelorarbeit schreiben?"
filter_6 = FILTERS[6]

In [26]:
docs = token_based_client.similarity_search_with_score(query=frage_6_vereinfacht, k=10)

2025-03-16 13:18:08,139 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


In [27]:
docs

[(Document(metadata={'id': 552, 'studiengang': 'Elektrotechnik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/prfungsundnderungsordnungen/fbeundi/FPO_BA_ELH_24.pdf', 'hash': '621298a5e40624ca5eddcbd61680aa4ab463ac15c2d5dd383956ba5cff90f6eb', '_id': 'b2c5ca10-4957-4d4e-b430-5ce324baa5f3', '_collection_name': 'mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks'}, page_content='en Kriterien, die eine eindeutige Abgrenzung ermöglichen, deutlich unterscheidbar und bewertbar ist und die Anforderungen nach Absatz 1 erfüllt. - (4) Die Bearbeitungszeit (Zeitraum von der Ausgabe bis zur Abgabe der Ausarbeitung) orientiert sich an der Modullänge und darf ein Semester nicht überschreiten. - (5) Die Beurteilung eines Portfolios erfolgt auf Grund der schriftlichen Ausarbeitungen und der mündlichen Prüfungen, sofern solche im Portfolio enthalten sind. Die Portfolioprüfung wird in der Regel von einer oder einem Prüfenden bewertet. Bei Kla